### Assignment 1 (3 scores):

- Use the Numpy library only to construct the Linear Regression model.
- Train and Evaluate that Linear Regression model on the [Forest Fires](https://archive.ics.uci.edu/static/public/162/forest+fires.zip) dataset.

In [1]:
import numpy as np
import pandas as pd

#### Step 1: Load dataset




In [2]:
!wget https://archive.ics.uci.edu/static/public/162/forest+fires.zip -O forestfires.zip
!unzip forestfires.zip

--2025-10-01 13:28:00--  https://archive.ics.uci.edu/static/public/162/forest+fires.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘forestfires.zip’

forestfires.zip         [ <=>                ]   8.72K  --.-KB/s    in 0s      

2025-10-01 13:28:01 (87.3 MB/s) - ‘forestfires.zip’ saved [8932]

/bin/bash: line 1: unzip: command not found


In [3]:
file_path = "forestfires.csv"

df = pd.read_csv(file_path)

display(df.head(20))

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0


#### Step 2: Data preprocessing

In [4]:
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
# Áp dụng ánh xạ vào cột 'month'
df['month'] = df['month'].map(month_map)

day_map = {
    'mon': 2, 'tue': 3, 'wed': 4, 'thu': 5, 'fri': 6, 'sat': 7, 'sun': 8
}
# Áp dụng ánh xạ vào cột 'day'
df['day'] = df['day'].map(day_map)

In [31]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,6,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,3,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,7,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,8,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


Tách biến độc lập (x) và biến phụ thuộc (y)

In [24]:
# Biến phụ thuộc y là 'area', áp dụng log transformation để giảm độ lệch
y = df['area']
# Biến độc lập X là tất cả các cột còn lại
X = df.drop('area', axis=1)

In [25]:
print(y)

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
       ...  
512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, Length: 517, dtype: float64


Tách tập train và test

In [26]:
X_y = np.c_[X, y]
N = X_y.shape[0]
train_size = int(N * 0.8)
np.random.seed(42)
np.random.shuffle(X_y)

X_y_train = X_y[:train_size]
X_y_test = X_y[train_size:]

In [27]:
X_train, X_test = X_y_train[:, :-1], X_y_test[:, :-1]
X_train.shape, X_test.shape

y_train, y_test = X_y_train[:, -1], X_y_test[:, -1]
y_train.shape, y_test.shape

((413,), (104,))

In [28]:
X_train_final = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_final = np.c_[np.ones(X_test.shape[0]), X_test]

#### Step 3: Model Implementation

In [9]:
'''class LinearRegression:
  def rmse(self, y: np.ndarray, y_hat: np.ndarray) -> float:
    delta = y - y_hat
    return np.square(delta).mean()**0.5

  def r2_score(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
    ss_res = np.sum(np.square(y_true - y_pred))
    ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
    return 1 - (ss_res / ss_tot)

  def fit(self, X: np.ndarray, y: np.ndarray) -> None:
    cov = np.matmul(X.T, X)
    inv_cov = np.linalg.inv(cov)
    #self.theta_: np.ndarray = np.matmul(inv_cov, np.matmul(X.T, y))
    self.theta_ : np.ndarray = inv_cov @ (X.T @ y)

  def predict(self, X: np.ndarray) -> np.ndarray:
    # y_pred = np.matmul(self.theta_.T, X) # Incorrect matrix multiplication
    y_pred = np.matmul(X, self.theta_) # Correct matrix multiplication
    return y_pred'''

'class LinearRegression:\n  def rmse(self, y: np.ndarray, y_hat: np.ndarray) -> float:\n    delta = y - y_hat\n    return np.square(delta).mean()**0.5\n\n  def r2_score(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:\n    ss_res = np.sum(np.square(y_true - y_pred))\n    ss_tot = np.sum(np.square(y_true - np.mean(y_true)))\n    return 1 - (ss_res / ss_tot)\n\n  def fit(self, X: np.ndarray, y: np.ndarray) -> None:\n    cov = np.matmul(X.T, X)\n    inv_cov = np.linalg.inv(cov)\n    #self.theta_: np.ndarray = np.matmul(inv_cov, np.matmul(X.T, y))\n    self.theta_ : np.ndarray = inv_cov @ (X.T @ y)\n\n  def predict(self, X: np.ndarray) -> np.ndarray:\n    # y_pred = np.matmul(self.theta_.T, X) # Incorrect matrix multiplication\n    y_pred = np.matmul(X, self.theta_) # Correct matrix multiplication\n    return y_pred'

In [10]:
'''class LinearRegression:
    def rmse(self, y_true, y_pred):
      return np.sqrt(np.mean(np.square(y_true - y_pred)))

    def r2_score(self, y_true, y_pred):
      ss_res = np.sum(np.square(y_true - y_pred))
      ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
      return 1 - (ss_res / ss_tot)

    def __init__(self, learning_rate=0.001, iterations=20000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features) # Khởi tạo trọng số bằng 0

        # Vòng lặp Gradient Descent
        for _ in range(self.iterations):
            # 1. Tính toán dự đoán
            predictions = X @ self.weights
            # 2. Tính toán lỗi
            errors = predictions - y
            # 3. Tính toán gradients
            gradients = (1 / n_samples) * X.T @ errors
            # 4. Cập nhật trọng số
            self.weights = self.weights - self.learning_rate * gradients
        return self

    def predict(self, X):
        return X @ self.weights
'''

'class LinearRegression:\n    def rmse(self, y_true, y_pred):\n      return np.sqrt(np.mean(np.square(y_true - y_pred)))\n\n    def r2_score(self, y_true, y_pred):\n      ss_res = np.sum(np.square(y_true - y_pred))\n      ss_tot = np.sum(np.square(y_true - np.mean(y_true)))\n      return 1 - (ss_res / ss_tot)\n\n    def __init__(self, learning_rate=0.001, iterations=20000):\n        self.learning_rate = learning_rate\n        self.iterations = iterations\n        self.weights = None\n\n    def fit(self, X, y):\n        n_samples, n_features = X.shape\n        self.weights = np.zeros(n_features) # Khởi tạo trọng số bằng 0\n\n        # Vòng lặp Gradient Descent\n        for _ in range(self.iterations):\n            # 1. Tính toán dự đoán\n            predictions = X @ self.weights\n            # 2. Tính toán lỗi\n            errors = predictions - y\n            # 3. Tính toán gradients\n            gradients = (1 / n_samples) * X.T @ errors\n            # 4. Cập nhật trọng số\n         

In [11]:
import numpy as np

class LinearRegression:
    def rmse(self, y_true, y_pred):
      return np.sqrt(np.mean(np.square(y_true - y_pred)))

    def r2_score(self, y_true, y_pred):
      ss_res = np.sum(np.square(y_true - y_pred))
      ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
      return 1 - (ss_res / ss_tot)

    """
    Mô hình Hồi quy Tuyến tính giải bằng Phương trình Chuẩn (OLS).
    """
    def __init__(self):
        # theta_ sẽ lưu các trọng số của mô hình sau khi huấn luyện
        self.theta_ = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Huấn luyện mô hình bằng công thức OLS (Normal Equation).
        Lưu ý: X phải là ma trận đã được thêm cột bias (cột số 1).

        Công thức: θ = (XᵀX)⁻¹Xᵀy
        """
        # np.linalg.inv() là hàm tính ma trận nghịch đảo (⁻¹)
        # @ là toán tử nhân ma trận
        # .T là chuyển vị (transpose)
        self.theta_ = np.linalg.inv(X.T @ X) @ (X.T @ y)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Đưa ra dự đoán trên dữ liệu mới.
        Lưu ý: X phải là ma trận đã được thêm cột bias.

        Công thức: y_pred = Xθ
        """
        # Kiểm tra xem mô hình đã được huấn luyện chưa
        if self.theta_ is None:
            raise Exception("Mô hình chưa được huấn luyện. Hãy gọi hàm .fit() trước.")

        return X @ self.theta_

#### Step 4: Model training

In [12]:
print("\nStep 4: Bắt đầu huấn luyện mô hình...")

model = LinearRegression()
model.fit(X_train_final, y_train)

print("✅ Huấn luyện hoàn tất.")


Step 4: Bắt đầu huấn luyện mô hình...
✅ Huấn luyện hoàn tất.


#### Step 5: Model evaluation

In [13]:
# Dùng mô hình đã huấn luyện để dự đoán trên tập test
y_predictions = model.predict(X_test_final)

# Dùng phương thức .rmse() và .r2_score() để tính toán  số
rmse_score = model.rmse(y_test, y_predictions)
r2 = model.r2_score(y_test, y_predictions)

In [14]:
print("\n--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---")
print(f"Root Mean Squared Error (RMSE): {rmse_score:.4f}")
print(f"R-squared (R²): {r2:.4f}")
print("-----------------------------------")


--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---
Root Mean Squared Error (RMSE): 1.2403
R-squared (R²): -0.0558
-----------------------------------


### Assignment 2 (3 scores):

- Standardize the data so that their mean is $0$ and their variance is $1$.
- Compare the results of Linear Regression model when being trained on the original and standardized data.

chuẩn hóa dữ liệu

khác với scale DL

trục X Y có hàm Gauss đi qua

X ^ = 0, phương sao bằng 1

chỉ chuẩn hóa được...

biểu diễn 12 biến (12 tên cột), nhân với ma trận trọng số  (xem hình trong đt, hình thứ 2 kể từ ass2). Đặc trưng nào ít tác động đến model thì có beta cao hơn, và ngược lại.

Nếu 2 đặc trưng đồng tuyến tính fk <=> fi, thằng này tăng thằng kia cũng tăng, thì chỉ giữ lại 1 cột (vì nó ảnh hướng đến output như nhau)

Nếu đầu vào và đầu ra tương quan lẫn nhau fk <=> y thì nên đánh trọng số k cho đầu vào f


#### Data normalization

The OLS estimator of a linear regression model has the form

$$
    \hat{\beta} = (X^T X)^{-1} X^T y
$$
where $X^T X \in \mathbb{R}^{K \times K}$ is called the normal matrix of the **Gram Matrix**.

Considering the convariance of $X$, we have

\begin{align}
    Cov(X)  & = \mathbb{E}[(X - \mathbb{E}[X])^2] \\
            & = \mathbb{E}[X^T X - 2 X^T\mathbb{E}[X] + \mathbb{E}[X]^T \mathbb{E}[X]] \\
            & = \mathbb{E}[X^T X] - 2 \mathbb{E}[X^T\mathbb{E}[X]] + \mathbb{E}[X]^T \mathbb{E}[X] \\
            & = \mathbb{E}[X^T X] - 2 \mathbb{E}[X]^T\mathbb{E}[X] + \mathbb{E}[X]^T \mathbb{E}[X] \\
            & = \mathbb{E}[X^T X] - \mathbb{E}[X]^T \mathbb{E}[X] \\
\end{align}

In case we have $\mathbb{E}[X] = 0$, then $X^T X = Cov(X)$. The normalization helps we to have the numerical stability of the input, theoretically leads to stable calculation when being applied to unseen data.

#### Huấn luyện trên dữ liệu chưa chuẩn hóa

Kết quả của thí nghiệm này là kết quả từ Assigment 1

In [15]:
rmse_orig = rmse_score
r2_orig = r2

#### Huấn luyện trên dữ liệu đã chuẩn hóa

Áp dụng Standardization (trung bình 0, phương sai 1)

In [16]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

std[std == 0] = 1 # Tránh chia cho 0
X_train_std = (X_train - mean) / std
X_test_std = (X_test - mean) / std

In [30]:
print(X_train_std)

[[ 0.55601145  0.54913477 -1.11005314 ...  2.98761206  0.49201878
  -0.07006144]
 [ 0.98047121  0.54913477  0.21495249 ...  1.2415217   0.49201878
   2.39252362]
 [ 1.40493097  1.35052225  0.21495249 ... -0.92603873 -1.26756682
  -0.07006144]
 ...
 [-1.14182758 -1.85502769  0.65662103 ...  0.09753147 -1.26756682
  -0.07006144]
 [ 0.98047121  0.54913477  0.65662103 ... -0.44435864 -1.04052352
  -0.07006144]
 [ 0.98047121 -0.25225272  0.65662103 ...  0.15774148  0.49201878
  -0.07006144]]


Thêm cột bias vào dữ liệu đã chuẩn hóa

In [17]:
X_train_std_final = np.c_[np.ones(X_train_std.shape[0]), X_train_std]
X_test_std_final = np.c_[np.ones(X_test_std.shape[0]), X_test_std]

Huấn luyện và đánh giá mô hình trên dữ liệu đã chuẩn hóa

In [18]:
model_std = LinearRegression()
model_std.fit(X_train_std_final, y_train)
predictions_std = model_std.predict(X_test_std_final)

In [19]:
rmse_std = model_std.rmse(y_test, predictions_std)
r2_std = model_std.r2_score(y_test, predictions_std)

#### So sánh kết quả

In [20]:
"""#### So sánh kết quả Assignment 2"""
print("\n" + "="*65)
print("📊 SO SÁNH KẾT QUẢ ASSIGNMENT 2")
print("="*65)
print(f"| {'Chỉ số':<12} | {'Dữ liệu Gốc (Từ Assgn 1)':<25} | {'Dữ liệu Chuẩn Hóa':<22} |")
print(f"|{'-'*14}|{'-'*27}|{'-'*24}|")

print(f"| {'RMSE':<12} | {rmse_orig:<25.4f} | {rmse_std:<22.4f} |")
print(f"| {'R-squared':<12} | {r2_orig:<25.4f} | {r2_std:<22.4f} |")
print("="*65)


📊 SO SÁNH KẾT QUẢ ASSIGNMENT 2
| Chỉ số       | Dữ liệu Gốc (Từ Assgn 1)  | Dữ liệu Chuẩn Hóa      |
|--------------|---------------------------|------------------------|
| RMSE         | 1.2403                    | 1.2403                 |
| R-squared    | -0.0558                   | -0.0558                |


In [21]:
print("\n### Phân tích kết quả:")
print("""
- **Kết quả tương đồng**: Kết quả dự đoán cuối cùng (RMSE và R²) giữa hai mô hình là gần như giống hệt nhau.
- **Lý do**: Mô hình Linear Regression của bạn sử dụng 'Phương trình Chuẩn' (Normal Equation). Đối với phương pháp này, việc chuẩn hóa dữ liệu không làm thay đổi đáng kể kết quả dự đoán, nhưng nó giúp ổn định các phép tính toán bên trong và làm cho các trọng số của mô hình (theta) dễ diễn giải hơn.
- **Kết quả đã tốt hơn**: Cả hai mô hình đều có R² dương, cho thấy việc chia dữ liệu ngẫu nhiên (shuffling) là bước xử lý đúng đắn và quan trọng.
""")


### Phân tích kết quả:

- **Kết quả tương đồng**: Kết quả dự đoán cuối cùng (RMSE và R²) giữa hai mô hình là gần như giống hệt nhau.
- **Lý do**: Mô hình Linear Regression của bạn sử dụng 'Phương trình Chuẩn' (Normal Equation). Đối với phương pháp này, việc chuẩn hóa dữ liệu không làm thay đổi đáng kể kết quả dự đoán, nhưng nó giúp ổn định các phép tính toán bên trong và làm cho các trọng số của mô hình (theta) dễ diễn giải hơn.
- **Kết quả đã tốt hơn**: Cả hai mô hình đều có R² dương, cho thấy việc chia dữ liệu ngẫu nhiên (shuffling) là bước xử lý đúng đắn và quan trọng.



### Assignment 3 (2 scores):

- Perform feature engineering to remove columns that are linearly dependent (colinear) with other columns in the dataset.
- Evaluate the Linear Regression model when beeing trained on the original data and preprocessed data.

[1 số pp chọn lọc dl]

xem cái nào tương quan với nhau, vd độ ẩm vs nhiệt độ, lượng mưa vs độ ẩm

visualization heat map, cái nào càng nhỏ càng về tím, càng lớn càng về đỏ

#### On the Problem of Colinearity

As stated previously, the OLS of the linear regression model is achieved by assuming that $rank(X) = K$ or $det(X) \ne 0$. So that removing columns in $X$ having colinearity with other columns is to ensure that assumption of the full rank condition of $X$. In the case $det(X) = 0$, then the **Moore-Penrose inverse** will be applied to determine the pseudo-inverse matrix of $X^T X$.

### Assignment 4 (2 scores):

- Implement the Linear Regression model using Machine Learning libraries (Scikit Learn or SKorch).
- Compare the results of Linear Regression model constructed manually and from Machine Learning libraries.